https://youtu.be/0ZqeBEa_MWo?si=euGFINmBNnY8qPbn&t=149

In [ ]:
import copy

import holoviews as hv
import numpy as np
import pandas as pd
from icecream import ic

In [ ]:
hv.extension("bokeh")

In [ ]:
spool_radius = 5.75
gear_ratio = (59.0 / 17.0) ** 2
STEP_DIVISION = 1
motor_steps_per_revolution = 100 * STEP_DIVISION
spool_circumfrence = spool_radius * 2 * np.pi
ic(spool_circumfrence)
steps_per_mm = motor_steps_per_revolution * gear_ratio / spool_circumfrence
ic(steps_per_mm)
max_rpm = 100.0
max_revs_per_second = max_rpm / 60.0
max_steps_per_second = max_revs_per_second * motor_steps_per_revolution
ic(max_steps_per_second)
max_velocity = max_steps_per_second / steps_per_mm
ic(max_velocity)
max_acceleration = 1.0
max_jerk = 1.0

In [ ]:
def s_curve(max_jerk, max_acceleration, max_velocity):
    """Solve minimal s curve that hits parameters"""
    sc = [{"t":0.0, "a": 0.0, "v":0.0, "p":0.0, "dt": 0.0, "da": 0.0, "dv": 0.0, "dp": 0.0} for i in range(7)]
    # either max_jerk to max_acceleration or max_velocity
    sc[0]["dt"] = sc[0]["t"] = min(ic(max_acceleration / max_jerk), ic(np.sqrt(max_velocity * 2 / max_jerk)))
    sc[0]["dv"] = sc[0]["v"] = max_jerk * sc[0]["t"] ** 2 / 2
    ic(1/2*sc[0]["dv"] * sc[0]["dt"] ** 2)
    sc[0]["dp"] = sc[0]["p"] = max_jerk * sc[0]["t"] ** 3 / 6
    
    # todo: finish stages
    # todo: use this to generate both full curve and minimal, 4 stage curve
    # make sanity check function

# Stage 0: Constant Jerk

Reach maximum acceleration

acceleration = max_jerk * t

velocity = 0.5 * max_jerk * t ** 2

t_max_jerk_2 = sqrt(velocity * 2 / max_jerk)

position = 1 /6 * max_jerk * t ** 3

In [ ]:
sc = [{"t":0.0, "v":0.0, "p":0.0, "dt": 0.0, "dv": 0.0, "dp": 0.0} for i in range(7)]
# either max_jerk to max_acceleration or max_velocity
sc[0]["dt"] = sc[0]["t"] = min(ic(max_acceleration / max_jerk), ic(np.sqrt(max_velocity * 2 / max_jerk)))
sc[0]["dv"] = sc[0]["v"] = max_jerk * sc[0]["t"] ** 2 / 2
ic(1/2*sc[0]["dv"] * sc[0]["dt"] ** 2)
sc[0]["dp"] = sc[0]["p"] = max_jerk * sc[0]["t"] ** 3 / 6
ic(sc[0])

# Stage 1: Constant Acceleration

Boost speed to almost maximum at max acceleration

delta_acc = max_velocity - v_max_jerk * 2

v_max_acc = v_max_jerk + max_acceleration * t_max_acc

p_max_acc = v_max_jerk * t_max_acc + 1 /2 * max_acceleration * t_max_acc ** 2

In [ ]:
sc[1]["dv"] = ic(ic(max_velocity) - ic(sc[0]["v"] * 2))
sc[1]["v"] = sc[0]["v"] + sc[1]["dv"]
sc[1]["dt"] = sc[1]["dv"] / max_acceleration
sc[1]["t"] = sc[0]["t"] + sc[1]["dt"]
sc[1]["dp"] = ic(sc[0]["v"] * sc[1]["dt"]) + ic(1 /2 * max_acceleration * sc[1]["dt"] ** 2)
sc[1]["p"] = sc[0]["p"] + sc[1]["dp"]
ic(sc[1])

# Stage 2: Constant jerk

Now we need to reduce our acceleration back down to 0 with out final velocity being max_velocity

j = -max_jerk

a = max_acc -max_jerk * t

v = v1 + max_acc * t - 1/2 * max_jerk * t**2

p = p1 + v1 * t + 1/2 * max_acc * t ** 2 - 1/6*max_jerk * t ** 3

In [ ]:
# All the constant jerk sections should have the same dt
sc[2]["dt"] = sc[0]["dt"]
sc[2]["t"] = sc[1]["t"] + sc[2]["dt"]
sc[2]["dv"] = sc[0]["dv"]
sc[2]["v"] = sc[1]["v"] + sc[2]["dv"]
sc[2]["dp"] = sc[1]["v"] * sc[2]["dt"] + 1 / 2 * max_acceleration * sc[2]["dt"] ** 2 - 1 / 6 * max_jerk * sc[2]["dt"] ** 3
sc[2]["p"] = sc[1]["p"] + sc[2]["dp"]
ic(sc[2])
assert(sc[2]["v"] == max_velocity)

# Stage 3: Coast

We can run a maximum velocity now

In [ ]:
# just setting dt to 0 for minimal curve
sc[3]["dt"] = 0
sc[3]["t"] = sc[2]["t"] + sc[3]["dt"]
sc[3]["dv"] = 0
sc[3]["v"] = sc[2]["v"]
sc[3]["dp"] = sc[3]["v"] * sc[3]["dt"]
sc[3]["p"] = sc[2]["p"] + sc[3]["dp"]
ic(sc[3])
assert(sc[3]["v"] == max_velocity)

# Stage 4: Constant Jerk

Change acceleration from 0 to -max_acceleration

j = -max_jerk

a = -max_jerk * t

v = max_vel - 1 / 2 * max_jerk * t ** 2

p = p3 + max_vel * t - 1 / 6 * max_jerl * t ** 3

In [ ]:
# All the constant jerk sections should have the same dt
sc[4]["dt"] = sc[0]["dt"]
sc[4]["t"] = sc[3]["t"] + sc[4]["dt"]
sc[4]["dv"] = -max_acceleration * sc[4]["dt"] ** 2 / 2
sc[4]["v"] = sc[3]["v"] + sc[4]["dv"]
sc[4]["dp"] = sc[3]["v"] * sc[4]["dt"] - 1 / 6 * max_jerk * sc[4]["dt"] ** 3
sc[4]["p"] = sc[3]["p"] + sc[4]["dp"]
ic(sc[4])

# Stage 5: Constant Acceleration

Reduce speed to almost 0

In [ ]:
# this will match the other constant acceleration stage
sc[5]["dv"] = -sc[1]["dv"]
sc[5]["v"] = sc[4]["v"] + sc[5]["dv"]
sc[5]["dt"] = sc[1]["dt"]
sc[5]["t"] = sc[4]["t"] + sc[5]["dt"]
sc[5]["dp"] = sc[5]["v"] * sc[5]["dt"] + 1 /2 * max_acceleration * sc[5]["dt"] ** 2
sc[5]["p"] = sc[4]["p"] + sc[5]["dp"]
ic(sc[5])

# Stage 6: Constant Jerk

Change acceleration from -max_acceleration to 0

In [ ]:
# All the constant jerk sections should have the same dt and dv
sc[6]["dt"] = sc[0]["dt"]
sc[6]["t"] = sc[5]["t"] + sc[6]["dt"]
sc[6]["dv"] = -sc[0]["dv"]
sc[6]["v"] = sc[5]["v"] + sc[6]["dv"]
assert(sc[6]["v"] == 0)
sc[6]["dp"] = sc[0]["dp"]
sc[6]["p"] = sc[5]["p"] + sc[6]["dp"]
ic(sc[6])

In [ ]:
pd.DataFrame(sc)

In [ ]:
hv.Curve(pd.DataFrame(sc), "t", "v") + hv.Curve(pd.DataFrame(sc), "t", "p")

# Sanity Check

Solve the curve segments from sc[i]["dt"] incrementally

In [ ]:
# section 0
dt = 0.00001
state = {
    "t": 0,
    "v": 0,
    "a": 0,
    "p": 0
}
hist = [state.copy()]
while True:
    state["t"] += dt
    state["a"] += max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    hist.append(state.copy())
    if state["t"] >= sc[0]["t"]:
        break

ic(hist[-1])
ic(sc[0])
ic(sc[0]["p"] - hist[-1]["p"])

In [ ]:
# section 1
state = {
    "t": sc[0]["t"],
    "v": sc[0]["v"],
    "a": max_acceleration,
    "p": sc[0]["p"]
}
while True:
    state["t"] += dt
    #state["a"] += max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    hist.append(state.copy())
    if state["t"] >= sc[1]["t"]:
        break

ic(hist[-1])
ic(sc[1])
ic(sc[1]["p"] - hist[-1]["p"])

In [ ]:
# section 2
state = {
    "t": sc[1]["t"],
    "v": sc[1]["v"],
    "a": max_acceleration,
    "p": sc[1]["p"]
}
while True:
    state["t"] += dt
    state["a"] -= max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    hist.append(state.copy())
    if state["t"] >= sc[2]["t"]:
        break

ic(hist[-1])
ic(sc[2])
ic(sc[2]["p"] - hist[-1]["p"])

In [ ]:
# section 3: coast
state = {
    "t": sc[2]["t"],
    "v": sc[2]["v"],
    "a": 0,
    "p": sc[2]["p"]
}
while True:
    state["t"] += dt
    #state["a"] -= max_jerk * dt
    #state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    hist.append(state.copy())
    if state["t"] >= sc[3]["t"]:
        break

ic(hist[-1])
ic(sc[3])
ic(sc[3]["p"] - hist[-1]["p"])

In [ ]:
# section 4: constant jerk
state = {
    "t": sc[3]["t"],
    "v": sc[3]["v"],
    "a": 0,
    "p": sc[3]["p"]
}
while True:
    state["t"] += dt
    state["a"] -= max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    hist.append(state.copy())
    if state["t"] >= sc[4]["t"]:
        break

ic(hist[-1])
ic(sc[4])
ic(sc[4]["p"] - hist[-1]["p"])

In [ ]:
# section 5
state = {
    "t": sc[4]["t"],
    "v": sc[4]["v"],
    "a": -max_acceleration,
    "p": sc[4]["p"]
}
while True:
    state["t"] += dt
    #state["a"] += max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    hist.append(state.copy())
    if state["t"] >= sc[5]["t"]:
        break

ic(hist[-1])
ic(sc[5])
ic(sc[5]["p"] - hist[-1]["p"])

In [ ]:
# section 6: constant jerk
state = {
    "t": sc[5]["t"],
    "v": sc[5]["v"],
    "a": -max_acceleration,
    "p": sc[5]["p"]
}
while True:
    state["t"] += dt
    state["a"] += max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    hist.append(state.copy())
    if state["t"] >= sc[6]["t"]:
        break

ic(hist[-1])
ic(sc[6])
ic(sc[6]["p"] - hist[-1]["p"])

In [ ]:
hv.Curve(hist, "t", "v") + hv.Curve(hist, "t", "p")

In [ ]:
start = np.array([150.0, 200.0]).reshape(1,2)
end = np.array([0.0, 0.0]).reshape(1,2)

In [ ]:
def full_s_curve(start, end):
    """Solve for full s-curve and ignore joint constraints"""
    
    dist = np.linalg.norm(start - end)
    
    if dist < sc[6]["p"]:
        print("path too short")
    
    out = copy.deepcopy(sc)
    
    # update coast stage (3) to fill up remaining time
    out[3]["dp"] = dist - sc[6]["p"]
    out[3]["dt"] = out[3]["dp"] / out[3]["v"]
    for i in range(3,7):
        out[i]["p"] = out[i-1]["p"] + out[i]["dp"]
        out[i]["t"] = out[i-1]["t"] + out[i]["dt"]
        
    return out

In [ ]:
out = full_s_curve(start, end)

In [ ]:
hv.Curve(pd.DataFrame(out), "t", "v") + hv.Curve(pd.DataFrame(out), "t", "p")

In [ ]:
out[6]

# Shorter?

Ok that's great. But what happens if we want to go shorter than a full curve? The next version would be one with truncated full acceleration stages (1 and 5)

Make a medimum s curve

# Minimal Curve

We first need to solve the constant jerk only version of the curve with no coasting or max accleration stages

In [ ]:
sc_min = [{"t":0.0, "v":0.0, "p":0.0, "dt": 0.0, "dv": 0.0, "dp": 0.0} for i in range(4)]

In [ ]:
# either max_jerk to max_acceleration or max_velocity
sc_min[0]["dt"] = sc_min[0]["t"] = min(ic(max_acceleration / max_jerk), ic(np.sqrt(max_velocity * 2 / max_jerk)))
sc_min[0]["dv"] = sc_min[0]["v"] = max_jerk * sc_min[0]["t"] ** 2 / 2
sc_min[0]["dp"] = sc_min[0]["p"] = max_jerk * sc_min[0]["t"] ** 3 / 6
ic(sc_min[0])

In [ ]:
# dt is the same for all sc_min segments
for i in range(1,4):
    sc_min[i]["dt"] = sc_min[0]["dt"]

# dv changes sign
sc_min[1]["dv"] = sc_min[0]["dv"]
sc_min[2]["dv"] = sc_min[3]["dv"] = -sc_min[0]["dv"]

# add up dv and dt to get v and t
for i in range(1,4):
    sc_min[i]["v"] = sc_min[i - 1]["v"] + sc_min[i]["dv"]
    sc_min[i]["t"] = sc_min[i - 1]["t"] + sc_min[i]["dt"]

# compute values of dp

# stage 1 dp
# j = -max_jerk
# a = max_acceleration - max_jerk * t
# v = sc_min[0]["v"] + max_acceleration * t - 1 / 2 *max_jerk * t ** 2
# p = sc_min[0]["p"] + sc_min[0]["v"] * t + 1/2 * max_acceleration * t ** 2 - 1/6 * max_jerk * t ** 3

sc_min[1]["dp"] = sc_min[0]["v"] * sc_min[1]["dt"] + 1 / 2 * max_acceleration * sc_min[1]["dt"] ** 2 - 1 / 6 * max_jerk * sc_min[1]["dt"] ** 3

# stage 2 dp
# j = -max_jerk
# a = -max_jerk * t
# v = sc[1]["v"] - 1 / 2 * max_jerk * t ** 2
# p = sc[1]["p"] + sc[1]["v"] * t - 1 / 6 * max_jerk * t ** 3

sc_min[2]["dp"] = sc_min[1]["v"] * sc_min[2]["dt"] - 1 / 6 * max_jerk * sc_min[2]["dt"] ** 3

sc_min[3]["dp"] = sc_min[2]["v"] * sc_min[3]["dt"] - 1 / 2 * max_acceleration * sc_min[3]["dt"] ** 2 + 1 / 6 * max_jerk * sc_min[3]["dt"] ** 3


for i in range(1, 4):
    sc_min[i]["p"] = sc_min[i - 1]["p"] + sc_min[i]["dp"]

ic(sc_min)

In [ ]:
hv.Curve(sc_min, "t", "v") + hv.Curve(sc_min, "t", "p")

# Sanity check minimal curve

In [ ]:
# section 0
dt = 0.00001
state = {
    "t": 0,
    "v": 0,
    "a": 0,
    "p": 0
}
min_hist = [state.copy()]
while True:
    state["t"] += dt
    state["a"] += max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    min_hist.append(state.copy())
    if state["t"] >= sc_min[0]["t"]:
        break

ic(min_hist[-1])
ic(sc_min[0])
ic(sc_min[0]["p"] - min_hist[-1]["p"])

In [ ]:
# section 1
state = {
    "t": sc_min[0]["t"],
    "v": sc_min[0]["v"],
    "a": max_acceleration,
    "p": sc_min[0]["p"]
}
while True:
    state["t"] += dt
    state["a"] += -max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    min_hist.append(state.copy())
    if state["t"] >= sc_min[1]["t"]:
        break

ic(min_hist[-1])
ic(sc_min[1])
ic(sc_min[1]["p"] - min_hist[-1]["p"])

In [ ]:
# section 2
state = {
    "t": sc_min[1]["t"],
    "v": sc_min[1]["v"],
    "a": 0,
    "p": sc_min[1]["p"]
}
while True:
    state["t"] += dt
    state["a"] -= max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    min_hist.append(state.copy())
    if state["t"] >= sc_min[2]["t"]:
        break

ic(min_hist[-1])
ic(sc_min[2])
ic(sc_min[2]["p"] - min_hist[-1]["p"])

In [ ]:
# section 3
state = {
    "t": sc_min[2]["t"],
    "v": sc_min[2]["v"],
    "a": -max_acceleration,
    "p": sc_min[2]["p"]
}
while True:
    state["t"] += dt
    state["a"] += max_jerk * dt
    state["v"] += state["a"] * dt
    state["p"] += state["v"] * dt
    min_hist.append(state.copy())
    if state["t"] >= sc_min[3]["t"]:
        break

ic(min_hist[-1])
ic(sc_min[3])
ic(sc_min[3]["p"] - min_hist[-1]["p"])

In [ ]:
hv.Curve(min_hist, "t", "v") + hv.Curve(min_hist, "t", "p")

In [ ]:
start = np.array([15.0, 20.0]).reshape(1,2)
end = np.array([0.0, 0.0]).reshape(1,2)

In [ ]:
dist = np.linalg.norm(start - end)

In [ ]:
upper_thresh = sc[6]["p"]
lower_thresh = sc[0]["dp"] * 4

In [ ]:
if dist > sc[6]["p"] || dist < 